In [ ]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import mean_squared_error

from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.compose import TransformedTargetRegressor
from sklearn.pipeline import FeatureUnion, Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.linear_model import LinearRegression

In [ ]:
df = pd.DataFrame(columns=['X1', 'X2', 'y'], data=[
                                                   [1,27,3],
                                                   [2,54,3.77976314968462],
                                                   [3,81,4.32674871092222],
                                                   [4,108,4.7622031559046],
                                                   [5,135,5.12992784003009]
                                                   ])
### y = Potencia1/3(X2)
# Esto asegura que un modelo de regresión lineal simple no pueda ajustarse perfectamente.

In [ ]:
df

,X1,X2,y
0,1,27,3.000000
1,2,54,3.779763
2,3,81,4.326749
3,4,108,4.762203
4,5,135,5.129928


In [ ]:
# Separamos en train y test

train = df.iloc[:2]
test = df.iloc[2:]

train_X = train.drop('y', axis=1)
train_y = train.y

test_X = test.drop('y', axis=1)
test_y = test.y

In [ ]:
test_X

,X1,X2
2,3,81
3,4,108
4,5,135


In [ ]:
#  verificacmos si la regresión lineal puede predecir correctamente

m1 = LinearRegression()
fit1 = m1.fit(train_X, train_y)
preds = fit1.predict(test_X)
print(f"\n{preds}")
print(f"RMSE: {np.sqrt(mean_squared_error(test_y, preds))}\n")


[4.5595263  5.33928945 6.1190526 ]
RMSE: 0.6746806117746605



In [ ]:
# Las predicciones no son malas, pero hagamos algunos cálculos sobre las características de entrada para mejorar
# ¿Qué pasa si sacamos la raíz cuadrada de X2 y multiplicamos por 2?

train_X.X2 = 3*np.cbrt(train_X.X2)

test_X.X2 = 3*np.cbrt(test_X.X2)

print(test_X)

m2 = LinearRegression()

fit2 = m2.fit(train_X, train_y)

preds = fit2.predict(test_X)

print(f"\n{preds}")

print(f"RMSE: {np.sqrt(mean_squared_error(test_y, preds))}\n")

   X1         X2
2   3  12.980246
3   4  14.286609
4   5  15.389784

[4.36271405 4.85136597 5.28275273]
RMSE: 0.10424164588074748



In [ ]:
# Restauremos los datos a sus valores originales y hagámoslo mediante transformadores personalizados mediante canalización.

train = df.iloc[:2]
test = df.iloc[2:]

train_X = train.drop('y', axis=1)
train_y = train.y

test_X = test.drop('y', axis=1)
test_y = test.y

In [ ]:
# Creamos una clase transformadora

#__init__: este es el constructor. Se llama cuando se inicializa el pipeline.
#fit(): Se llama cuando hacemos fit en el pipeline.
#transform(): se llama cuando usamos fit o transform en el pipeline

class ExperimentalTransformer(BaseEstimator, TransformerMixin):
  def __init__(self):
    print('\n>>>>>>>init() called.\n')

  def fit(self, X, y = None):
    print('\n>>>>>>>fit() called.\n')
    return self

  def transform(self, X, y = None):
    print('\n>>>>>>>transform() called.\n')
    X_ = X.copy() # creamos una copia para evitar cambios en el conjunto de datos original
    X_.X2 = 3 * np.cbrt(X_.X2)
    return X_

In [ ]:
# sin transformación de entrada - para validar que obtenemos los mismos resultados que antes

print("create pipeline 1")
pipe1 = Pipeline(steps=[
                       ('linear_model', LinearRegression())
])

print("fit pipeline 1")
pipe1.fit(train_X, train_y)

print("predict via pipeline 1")
preds1 = pipe1.predict(test_X)

print(f"\n{preds1}")  # should be [13.72113586 16.93334467]
print(f"RMSE: {np.sqrt(mean_squared_error(test_y, preds1))}\n")

create pipeline 1
fit pipeline 1
predict via pipeline 1

[4.5595263  5.33928945 6.1190526 ]
RMSE: 0.6746806117746605



In [ ]:
# con transformation de entradas

print("create pipeline 2")
pipe2 = Pipeline(steps=[
                       ('experimental_trans', ExperimentalTransformer()),    # esto dispara una llamda a __init__
                       ('linear_model', LinearRegression())
])

# una sintaxis alternativa y más corta para hacer lo anterior, sin nombrar cada paso, es:
#pipe2 = make_pipeline(ExperimentalTransformer(), LinearRegression())

print("fit pipeline 2")
pipe2.fit(train_X, train_y)

print("predict via pipeline 2")
preds2 = pipe2.predict(test_X)

print(f"\n{preds2}")  # should be [14. 17.]
print(f"RMSE: {np.sqrt(mean_squared_error(test_y, preds2))}\n")



# a. __init__ se llama cuando inicializamos pipe2.

# b. fit() y transform() se llaman cuando llamamos fit() con los datos de entrenamiento.
# Esto es necesario para transformar las características de entrada mientras se intenta predecir train_y.

# c. transform() se llama  cuando usamos predict(test_X): las características de prueba de entrada deben tener raíz cuadrada y duplicarse también antes de hacer predicciones.

create pipeline 2

>>>>>>>init() called.

fit pipeline 2

>>>>>>>fit() called.


>>>>>>>transform() called.

predict via pipeline 2

>>>>>>>transform() called.


[4.36271405 4.85136597 5.28275273]
RMSE: 0.10424164588074748

